## Project luther 

In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pandas as pd

## Scraping list of cities (Wikipedia)

In [2]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')

soup_cities = BeautifulSoup(response.text, "lxml")

In [3]:
tables=soup_cities.find_all("table")
table_cities = tables[4]

wiki_data = []
cities = [city for city in table_cities.find_all('tr')]
wiki_columns = cities[0]
cities = cities[1:]

#Titles

ciudad = []

for city in cities:
    Data_city = []
    
    #City name
#    ciudad.append(city.find_all('td')[1].find('a')['title'].split(",")[0])
    Data_city.append(city.find_all('td')[1].find('a')['title'].split(",")[0])
    #State
    if (city.find_all('td')[2].find('a')) != None:
        Data_city.append(city.find_all('td')[2].find('a')['title'])
    else:
        Data_city.append(str(city.find_all('td')[2]).split("</span>")[1][:-6])
    #Census estimate 2017
    Data_city.append(str(city.find_all('td')[3].renderContents())[2:-3])
    #Area 
    Data_city.append(str(city.find_all('td')[7].renderContents())[2:-25])
    #Density
    Data_city.append(str(city.find_all('td')[9].renderContents())[2:-18])
    wiki_data.append(Data_city)


df_cities = pd.DataFrame(wiki_data)

In [4]:
col_titles = ["City", "State", "Population_2017", "Area", "Pop_density"]
df_cities.columns = col_titles

## Scraping Tripadvisor
The following code makes the scraping of the cities already collected from wikipedia in the previous cells

In [5]:
def ex_things_todo(city, state, sleep):
    things_to_do =[]
    chromedriver = "/home/pnietoramos/chromedriver" 
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get("https://www.tripadvisor.com/Attractions")
    time.sleep(sleep)
    query = driver.find_element_by_class_name("typeahead_input")
    query.send_keys(city + ", " + state + ", USA")
    time.sleep(sleep)
    query = driver.find_element_by_id("SUBMIT_THINGS_TO_DO")
    query.send_keys(Keys.RETURN)
    time.sleep(sleep)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    if len(soup.find_all("div", class_="popRanking wrap")) > 0:
        attract = soup.find_all("div", class_="popRanking wrap")[0].renderContents()
        things_to_do.append(str(attract).split()[2])
    else:
        things_to_do.append("NaN")
    driver.close()
    return things_to_do

In [6]:
cities = df_cities["City"]
states = df_cities["State"]

In [7]:
things_to_do =[]
for i in range(len(cities)):
    things_to_do.append(ex_things_todo(cities[i], states[i], 1))

In [9]:
df_things = pd.DataFrame(things_to_do)

In [11]:
df_things.head()

,0
0,"1,192"
1,624
2,698
3,332
4,239


In [12]:
#Droping empy cells
df_cities["Things_to_do"] = df_things[:]
len(df_cities[df_cities["Things_to_do"]=="NaN"])

1

In [13]:
df_cities.to_csv('Cities_Things.csv')

## Median rent
Csv downloaded from:

https://www.apartmentlist.com/rentonomics/rental-price-data/

In [4]:
df_rent = pd.read_csv("Apartment-List-Rent-Data-Summary_2018-6.csv")

In [5]:
df_rent.head(1)

,Location,Location_Type,Year,Month,Price_Studio,Price_1br,Price_2br,Price_3br,Price_4br,MoM_Price_Change,YoY_Price_Change
0,National,National,2018,6,830,950,1178,1583,1844,0.4,1.4


In [6]:
df_rent = df_rent[df_rent["Location_Type"]=="City"]
df_rent = df_rent.drop(["Location_Type","Year","Month","Price_Studio","Price_2br","Price_3br","Price_4br","MoM_Price_Change","YoY_Price_Change"], axis=1)
df_rent.head(1)

,Location,Price_1br
52,"Aberdeen, MD",956


In [7]:
df_rent["State_code"] = df_rent["Location"].apply(lambda x: x.split(", ")[1])
df_rent["Location"] = df_rent["Location"].apply(lambda x: x.split(",")[0])
df_rent["City"] = df_rent["Location"]
df_rent = df_rent.drop(["Location"], axis = 1)
df_rent.head(1)

,Price_1br,State_code,City
52,956,MD,Aberdeen


## Importing state codes and using them for the cities
To match rent information data set with census and trip advisor I need the state codes.  

In [8]:
response = requests.get('https://www.ups.com/worldshiphelp/WS16/ENU/AppHelp/Codes/State_Province_Codes.htm')

soup_states = BeautifulSoup(response.text, "lxml")

In [9]:
state_to_code = {}
table_states = soup_states.find_all("table")[0] 
states = [line for line in table_states.find_all('tr')[1:]]
for state in states:
    state_to_code[str(state.find_all('p')[0].renderContents()).split(",")[0][2:]] =str(state.find_all('p')[1].renderContents())[2:-1]

In [10]:
df_for_add = pd.read_csv("Cities_Things_FINAL.csv")
df_for_add["State"] = df_for_add["State"].apply(lambda x: x.split("(")[0])
df_for_add.head(1)

,Unnamed: 0,City,State,Population_2017,Area,Pop_density,Things_to_do
0,0,New York City,New York,"8,622,698",780.9,"10,933","1,192"


In [11]:
df_for_add["State_code"] = df_for_add["State"].apply(lambda x: state_to_code[str(x.split(",")[0].strip())])
df_for_add.head(1)

,Unnamed: 0,City,State,Population_2017,Area,Pop_density,Things_to_do,State_code
0,0,New York City,New York,"8,622,698",780.9,"10,933","1,192",NY


In [12]:
merged_df = df_for_add.merge(df_rent, how = 'inner', on = ["City", "State_code"])
merged_df.head(20)

,Unnamed: 0,City,State,Population_2017,Area,Pop_density,Things_to_do,State_code,Price_1br
0,1,Los Angeles,California,"3,999,759","1,213.9","3,276",624,CA,1361
1,2,Chicago,Illinois,"2,716,450",588.7,"4,600",698,IL,1079
2,3,Houston,Texas,"2,312,717","1,651.1","1,395",332,TX,841
3,4,Phoenix,Arizona,"1,626,078","1,340.6","1,200",239,AZ,836
4,5,Philadelphia,Pennsylvania,"1,580,863",347.6,"4,511",483,PA,972
5,6,San Antonio,Texas,"1,511,946","1,194.0","1,250",282,TX,839
6,7,San Diego,California,"1,419,516",842.3,"1,670",463,CA,1554
7,8,Dallas,Texas,"1,341,075",882.9,"1,493",262,TX,893
8,9,San Jose,California,"1,035,317",459.7,"2,231",89,CA,2082
9,10,Austin,Texas,"950,715",809.9,"1,170",347,TX,1137


In [18]:
merged_df.to_csv("Cities_things_rent.csv")

## Income per household and crime rates (http://www.city-data.com/)

In [25]:
city_data =[]
cities = list(merged_df["City"])
states = list(merged_df["State"])

In [26]:
#Get to the page and select the city
def ex_income(city, state, sleep):
    income_single= []
    chromedriver = "/home/pnietoramos/chromedriver" 
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get("http://www.city-data.com/")
    time.sleep(sleep)
    query = driver.find_element_by_id("intelligent_search")
    query.send_keys(city+","+state+"\r\n")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    income_single.append(city)
    income_single.append(state)
    if len(soup.find_all('section', class_='median-income')) > 0:
        income_single.append(str(soup.find_all('section', class_='median-income')[0]).split("</b>")[1].split()[0][1:])
    else:
        income_single.append("NaN")
    if len(soup.find_all('table', class_='crime tabBlue table')) > 0:
        income_single.append(str(soup.find_all('table', class_='crime tabBlue table')[0].find_all('tr',class_="norm2")[0].find_all('td')[-1].renderContents())[2:-1])
        income_single.append(str(soup.find_all('table', class_='crime tabBlue table')[0].find_all('tr',class_="norm2")[3].find_all('td')[-1].renderContents())[2:-1])
    else:
        income_single.append("NaN")
        income_single.append("NaN")
    driver.close()
    return income_single

In [27]:
income = []
for i in range(len(cities)):
    income.append(ex_income(cities[i], states[i], 1))

In [28]:
col_titles = ["City", "State", "median_income","murders_2016_pro10e5","assaults_2016_pro10e5"]
df_income = pd.DataFrame(income)
df_income.columns = col_titles
df_income

,City,State,median_income,murders_2016_pro10e5,assaults_2016_pro10e5
0,Renton,Washington,"71,951",1.1,154.8
1,Sparks,Nevada,"57,444",3.1,325.9
2,Allen,Texas,"102,064",0.0,43.6
3,Tuscaloosa,Alabama,"39,298",14.0,240.2
4,San Angelo,Texas,"46,619",5.9,248.4
5,Vacaville,California,"85,422",0.0,122.9


In [29]:
df_income.to_csv('Income_crime_FINAL_OOOOO.csv')